# Flow Routing Lesson 5.
You can run this notebook on your laptop with landlab v2.6.0 or in lab.openearthscape using the CSDMS environment.

Please first install the flow_router "component" experimental version by:
- opening a command line tool,
- going into the flow_router folder `cd landlab_flowrouting_clinic_2023/flow_router`, 
- activating an environment having the Cython package installed, in lab.openearthscape you can do this by keying `source activate dev`,
- and keying `python setup.py build_ext --inplace`

Then to test if the component is correctly installed do in the command line tool:
- `cd ..`
- `python`
- `from flow_router import FlowRouter`
- No error should be raised.
Presently (2023, May), install should be successful on Linux and Windows. Depending on your C/C++-compiler on Mac install can fail on that OS.

Then you can use the FlowRouter experimental component in the notebook.

## 1. Using the any-grid-type FlowRouter to solve depressions and direct and accumulate flow.

The FlowRouter calculates single-to-single flow directions and accumulations, and handles depression at the same time. Compared with the DepressionFinderAndRouter, the component works on NetworkModelGrids, is compatible with multiple flow, and uses a fast implementation of the priority flood flow router algorithm. That being said, due to the algorithm, the FlowRouter might underperform the DepressionFinderAndRouter for grids without depressions.

The FlowRouter doesn't wrap another external package, as the PriorityFloodFlowRouter does, and it consists of a python/landlab code, with a Cython part which requires to be compiled before use (this is what does the `pip install -e .` above). The FlowRouter has a lower execution time than the PriorityFloodFlowRouter for large grids and handles depressions on all types of gridm while the PriorityFloodFlowRouter handles depressions only in RasterModelGrids.

*(Priority Flood Flow algorithm and downstream to upstream node ordering adapted from Barnes et al., 2014 and Braun and Willett, 2013).*

- We start by importing the packages and the experimental FlowRouter component.

In [ ]:
# Libraries required
####################
# Python libraries
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.collections as mpl_collections
import time  # performance testing

# Landlab libraries/methods
# Grid management
from landlab import (
    RasterModelGrid,
    HexModelGrid,
    RadialModelGrid,
    FramedVoronoiGrid,
    VoronoiDelaunayGrid,
    NetworkModelGrid,
)

# Depression and flow handling
from landlab.components import (
    FlowDirectorSteepest,
    FlowDirectorD8,
    FlowDirectorMFD,
    FlowDirectorDINF,
    FlowAccumulator,
    DepressionFinderAndRouter,
    PriorityFloodFlowRouter,
)

# Flood status
from landlab.components.depression_finder.floodstatus import FloodStatus

# import landlab plotting functionality
from landlab import imshowhs_grid, imshow_grid
from landlab.plot.drainage_plot import drainage_plot
from landlab.plot import graph

# Experimental FlowRouter component. Requires that the code of the component be in a subfolder flow_router of the folder of this notebook.
from flow_router import FlowRouter

- We then setup the type, the size and the spacing of the grid. Let's choose a hexagonal grid and display the nodes (You could do the same with a RasterModelGrid):

In [ ]:
# Selection of the grid type
model_grid = HexModelGrid 

# Grid parameters
xy_spacing = (10, 10)
# Random generator to generate the coordinates of a VoronoiDelaunayGrid. To changeµ the coordinates,
# change the seed (but you'll have to change the coordinates of perimeter and base-level nodes in next cell)
random_generator = np.random.Generator(np.random.PCG64(seed=200))
grid_param = {
        "shape": (9, 5),
        "spacing": xy_spacing[0],
        "node_layout": ["hex", "rect"][0],
        "xy_axis_units": "m",
}

# Creation of the grid
g = model_grid(**grid_param)
nodes_n = g.number_of_nodes
# Create field topographic elevation at nodes with value 0 (will be modified later)
z = g.add_zeros("topographic__elevation", at="node")

if nodes_n < 200:
    graph.plot_graph(g, at="node")

- Let's now set the base-level and outlets.

    - Landlab grids consist of nodes (and other elements as cells, links, ...) which have 4 status. (1) Nodes that receives flow from other nodes and transfer flow to other nodes are **core** nodes. (2) and (3), Nodes that receive flow but don't transfer it are either **fixed value** or **fixed gradient**. These nodes form the base-level of the grid and can be outlets of the grid (and they can have different elevation depending on the location on the grid. (4) Nodes that neither receive nor transfer flow are **closed**.

    - By default, all grids except VoronoiDelaunayGrid and NetworkModelGrid have their base-level defined at their **perimeter**, which is also called **boundary**. In other words, the perimeter nodes have the status fixed value. The perimeter nodes of the NetworkModelGrid are by default all with status=core. So, if we choose NetworkModelGrid, it can be appropriate to specifically tell the code where the base-level nodes are, using the status fixed value and closed.

    - Here, we set all perimeter nodes are base-level except on one edge:

In [ ]:
if isinstance(g, RasterModelGrid):
    g.set_closed_boundaries_at_grid_edges(
        right_is_closed=False,
        top_is_closed=True,
        left_is_closed=False,
        bottom_is_closed=False,
    )
elif isinstance(g, HexModelGrid):
    g.status_at_node[g.nodes_at_top_edge] = g.BC_NODE_IS_CLOSED

- The base-level and closed nodes will be visualized on the grid by a specific display method ```drainage_plot()``` after running the flow directions. For RasterModelGrids only, the closed perimeter nodes can also be visualized by the black color using the method imshow_grid (see below).

- Let's now generate a random topography on which we'll direct and accumulate the flow. We fix topography of the node of id 0 at zero. And we create a wide depression in the grid. 

    - Note that it's also possible to direct and accumulate flow on other surface than the topographic elevation.

    - *To generate a different topography, you can modify* ```seed=500``` *by* ```seed=200``` *for instance, or remove the seed.*

In [ ]:
# set constant random seed for "stable" random pull
random_generator = np.random.Generator(np.random.PCG64(seed=500))
random_topography = 10.0  # range in elevation (from 0)
z = g.at_node["topographic__elevation"] = (
    random_topography * random_generator.random(nodes_n) + 1.0
)
z[0] = 0.0

# Add a large depression within the grid
depression_nodes = []
if isinstance(g, RasterModelGrid):
    depression_nodes = [43, 44, 45, 53, 54, 55, 63, 64, 65, 66, 74, 75]
elif isinstance(g, HexModelGrid):
    depression_nodes = [29, 30, 37, 38, 39, 40, 45, 46, 47]
z[depression_nodes] = random_generator.random(len(depression_nodes))

imshow_grid(
        g,
        "topographic__elevation",
        cmap="terrain",
        grid_units=("m", "m"),
        var_name="Elevation (m)",
        plot_name="Topographic elevation",
        show_elements=True,
    )

- Remarks:
    - For RasterModelGrids: in this display, you can see the perimeter nodes for the RasterModelGrid. The closed perimeter nodes are black-colored. <br>
    - For other types of grids: the perimeter nodes are hidden because of the implementation of the method ```imshow_grid()```. But for VoronoiDelaunayGrid, a few perimeter nodes can still be visible due to the incorrect determination of the perimeter nodes for this type of grid.

    - Note the blue-tainted depression.

- Let's now run the FlowRouter using single-to-single flow. 
    - Note that the FlowRouter is compatible with all types of grids and with single-to-single and single-to-multiple flow. We'll show that later.

In [ ]:
router = FlowRouter(g)
router.run_one_step()

drainage_plot(g, surf_cmap="terrain")
plt.show()
imshow_grid(
    g,
    "surface_water__discharge",
    cmap="Blues",
    grid_units=("m", "m"),
    var_name="Discharge (m^3/s)",
    plot_name="Surface water discharge",
    show_elements=True,
)


- Remarks:
    - The drainage is displayed by the ```drainage_plot()``` method (which is not implemented for NetworkModelGrid). The cells are colored according to their elevation (right colorbar), except for perimeter nodes of non RasterModelGrids. The flow is directed along the steepest slope (yellow arrows). Diagonals are included by default for RasterModelGrids. The final outlet of the catchments are always a base-level perimeter node, because the FlowRouter component breach depressions.
    - the DepressionFinderAndRouter works differently from the FlowRouter. The DepressionFinderAndRouter scans all the sinks. These sinks are provided by the FlowDirector/Accumulator (field ```flow__sink_flag```). The component then maps depressions and finds the lowest node at the perimeter of the depression. Once found, it considers this point as the depression outlet and recalculates all flow directions and accumulations by evacuating the depression.
    <br />
    <br />
    - The FlowRouter handles the depressions and calculates the single-to-single flow directions at the same time. For this, the component starts from the base-level nodes (with status fixed value or fixed gradient) and constructs a flow path by overcoming the depressions. It ensures that peaks remain peaks by always handling nodes with the lowest elevation first. By default for rasters, the component also diagonals (this option can be modified by the argument ```diagonals=False```).

    - The flows are directed differently from the DepressionFinderAndRouter because the algorithms are different. The FlowRouter algorithm starts from base-level nodes to construct the paths by simultaneously solving the depressions (this is the **priority flood flow algorithm**). Conversely, the FlowDirector calculates donors and receivers without starting from the base-level nodes and after that, the DepressionFinderAndRouter scans the depressions to solve them.

## 2. FlowRouter with multiple flows.
- Let's now try to combine the FlowRouter with multiple flow directions and accumulations. Note that multiple flow is not implemented for NetworkModelGrids.

- Let's delete the fields which need to be resized in the multiple flow configuration.

In [ ]:
# Delete fields used with different shapes in FlowDirectorMFD
for field in [
    "drainage_area",
    "flow__link_to_receiver_node",
    "flow__receiver_node",
    "flow__receiver_proportions",
    "flow__upstream_node_order",
    "outlet_node",
    "surface_water__discharge",
    "topographic__steepest_slope",
]:
    if field in g.at_node.keys():
        g.delete_field(loc="node", name=field)

- Now, we can run the FlowRouter in multiple_flow mode with the argument ```single_flow=False```, for which it will only produce a depression free elevation and other informations as the depth of the depression, the outlet of the depression and the flooded status.

In [ ]:
# We take into account diagonal links for RasterModelGrids but not for other grids
router = FlowRouter(g, single_flow=False)
router.run_one_step()

plt.show()
imshow_grid(
    g,
    "depression_free__elevation",
    cmap="terrain",
    grid_units=("m", "m"),
    var_name="Elevation (m)",
    plot_name="Depression free elevation",
    show_elements=True,
)

- Remarks:
    - The depression-free elevation is determined by starting from the elevation the outlet of the depression and going upstream. Depression-free elevation is calculated this way: depression-free elevation of the receiver + a small epsilon. This epsilon is there to create an artificial very low slope to direct the multiple flow.

    - This depression-free elevation surface is given as an input to the multiple flow directors.

- Let's now run the **FlowDirectorMFD**, a multiple flow director component implemented by adapting the Multiple Flow Direction (MFD) algorithm of [Quinn et al., 1991](https://doi.org/10.1002/hyp.3360050106).

In [ ]:
flow_director = "FlowDirectorMFD"
diagonals = True if isinstance(g, RasterModelGrid) else False

accumulator = FlowAccumulator(
    g,
    surface="depression_free__elevation",
    flow_director=flow_director,
    diagonals=diagonals,
)

accumulator.run_one_step()
drainage_plot(g, surface="depression_free__elevation", surf_cmap="terrain")
plt.show()
imshow_grid(
    g,
    "surface_water__discharge",
    cmap="Blues",
    grid_units=("m", "m"),
    var_name="Discharge (m^3/s)",
    plot_name="Surface water discharge",
    show_elements=True,
)

- Now, let's run the FlowDirectorDinf, a dual-flow director component implemented by adapting the D-Infinity (DINF) algorithm of [Tarboton, 1997](https://doi.org/10.1029/96WR03137). 
    - This component works independently from the FlowDirectorMFD.
    - Note that component only works for RasterGrids.

    - Again, we delete the fields which need to be resized in the multiple flow configuration and then we instantiate and run the component.

In [ ]:
# Delete fields used with different shapes in FlowDirectDINF
for field in [
    "drainage_area",
    "flow__link_to_receiver_node",
    "flow__receiver_node",
    "flow__receiver_proportions",
    "flow__upstream_node_order",
    "outlet_node",
    "surface_water__discharge",
    "topographic__steepest_slope",
]:
    if field in g.at_node.keys():
        g.delete_field(loc="node", name=field)

In [ ]:
flow_director = "FlowDirectorDINF"
if isinstance(g, RasterModelGrid):
    accumulator = FlowAccumulator(
        g, surface="depression_free__elevation", flow_director=flow_director
    )

    accumulator.run_one_step()
    drainage_plot(g, surface="depression_free__elevation", surf_cmap="terrain")
    imshow_grid(
        g,
        "surface_water__discharge",
        cmap="Blues",
        grid_units=("m", "m"),
        var_name="Discharge (m^3/s)",
        plot_name="Surface water discharge",
        show_elements=True,
    )

## 3. The FlowRouter in a Landscape Evolution Model.

- The FlowRouter may be convenient to run landscape evolution models (LEM) over large grids and a large number of iterations (because it's faster than the DrainageFinderAndRouter in this use case).

- Let's start by deleting the fields which shape is for multiple flow directions.

In [ ]:
# Delete fields used with different shapes
for field in [
    "drainage_area",
    "flow__link_to_receiver_node",
    "flow__receiver_node",
    "flow__receiver_proportions",
    "flow__upstream_node_order",
    "outlet_node",
    "surface_water__discharge",
    "topographic__steepest_slope",
]:
    if field in g.at_node.keys():
        g.delete_field(loc="node", name=field)

- Then, we construct a simple LEM. During 1 million years, the LEM will simulate an uplifting landscape from sea-level. Two erosion processes are included:
    - bedrock mass wasting: slopes higher than a critical threshold are progressively chop down to the critical threshold (```ThresholdEroder```)
    - river incision: rivers incise the bedrock with a stream power law (```StreamPowerEroder```).

    - We assume the system evacuates all sediment out of the grid.

In [ ]:
from tqdm import tqdm
from landlab.components import StreamPowerEroder, SpaceLargeScaleEroder, ThresholdEroder

uplift_rate = 0.00001  # m/y
dt = 1000  # timestep [y]
t = 1e6  # 1e6 # totale time [y]
n_dt = int(np.floor(t / dt))

router = FlowRouter(g)
landslider = ThresholdEroder(g)
# eroder = StreamPowerEroder(g, K_sp=1e-6, m_sp=0.5, n_sp=1.0, erode_flooded_nodes=False)
g.add_field("soil__depth", np.zeros(nodes_n), at="node", clobber=True)
g.add_field("bedrock__elevation", z, at="node", clobber=True)
eroder = SpaceLargeScaleEroder(g, K_br=8*1e-6, K_sed=1e-5)

for i in tqdm(range(n_dt)):
    z[g.status_at_node == g.BC_NODE_IS_CORE] += uplift_rate * dt
    router.run_one_step()
    landslider.run_one_step()
    router.run_one_step()
    eroder.run_one_step(dt)

In [ ]:
if nodes_n < 200:
    drainage_plot(g, surf_cmap="terrain")
else:
    imshow_grid(
        g,
        "topographic__elevation",
        cmap="terrain",
        grid_units=("m", "m"),
        var_name="Elevation (m)",
        plot_name="Topographic elevation",
    )
plt.show()
imshow_grid(
    g,
    "surface_water__discharge",
    cmap="Blues",
    grid_units=("m", "m"),
    var_name="Discharge (m^3/s)",
    plot_name="Surface water discharge",
)

<br>
<br/>
Please reuse this material citing Lenard et al., 2023, CSDMS Annual Meeting.<br/>
Author: Sebastien Lenard sebastien.lenard@gmail.com<br>
Date: 2023, May.

### Click here to learn about <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">Landlab tutorials</a>